In [41]:
# formulation lacks initial boundary conditions, doesnt work fully
using JuMP
import HiGHS

model = Model(HiGHS.Optimizer)

# PARAMETERS
num_trips = 5 # rows
num_stops = 5 # columns
dispatch_times = [4, 6, 8, 10, 12]

target_headway = [
    1 2 2 1 2
    1 2 2 1 2
    1 2 2 1 2
    1 2 2 1 2
  ]

interstation_travel_time = [
    1 2 3 4
    1 2 3 4
    1 2 3 4
    1 2 3 4
    1 2 3 4
  ]

arrival_rate = 0.5
alighting_percentage = [0.1 0.2 0.3 0.4 0.5]
boarding_time = 1
alighting_time = 2
stop_weights = [0.1 0.2 0.3 0.4 0.5]
num_buses = 5
bus_capacity = 100
initial_passengers = [0 0 0 0 0]


# DECISION VARIABLES
@variable(model, dispatch_offsets[1:num_trips] >= 0)
@variable(model, holding_durations[1:num_trips, 1:num_stops] >= 0)

# EXPRESSIONS

# arrival time js
@expression(
    model,
    arrival_time[j = 1:num_trips, s = 1:num_stops],
    (
        dispatch_times[j] +
        dispatch_offsets[j] +
        sum(interstation_travel_time[j,φ] for φ in 1:s-1) +
        sum(holding_durations[j,φ] for φ in 2:s-1)
    )
)

# headway js
@expression(
    model,
    headway[j = 2:num_trips, s = 1:num_stops],
    (
        arrival_time[j,s] - arrival_time[j-1,s]
    )
)

# number of passengers actually boarding js
@expression(
    model,
    num_boarding[j = 1:num_trips, s = 1:num_stops],
    (
        min(num_want_board[j,s], bus_capacity - busload[j,s] + num_alighting[j,s])
    )
)

# minimum dwell time js
@expression(
    model,
    minimum_dwell[j = 1:num_trips, s = 1:num_stops],
    (
        max(num_boarding[j,s] * boarding_time, num_alighting[j,s] * alighting_time)
    )
)

# busload js
@expression(
    model,
    busload[j = 1:num_trips, s = 1:num_stops],
    (
        busload[j,s-1] - num_alighting[j,s-1] + num_want_board[j,s-1]
    )
)

# number of passengers wanting to board js
@expression(
    model,
    num_want_board[j = 1:num_trips, s = 1:num_stops],
    (
        (1 + arrival_rate * boarding_time) * (arrival_rate * (headway[j,s] - minimum_dwell[j-1,s]) + stranded[j-1,s])
    )
)

# number of passengers alighting js
@expression(
    model,
    num_alighting[j = 1:num_trips, s = 1:num_stops],
    (
        round(busload[j,s] * alighting_percentage[s])
    )
)

# stranded passengers js
@expression(
    model,
    stranded[j = 1:num_trips, s = 1:num_stops],
    (
        num_want_board[j,s] - num_boarding[j,s]
    )
)
stranded[1] = initial passengers # initialisation

# CONSTRAINTS

# no overtaking
@constraint(
    model,
    no_overtaking_constraint,
    headway[j,s] >= 0 for j in 1:num_trips for s in 1:num_stops
)
# first and last trip cannot be optimised
@constraint(
    model,
    unoptimised_first_trip_constraint,
    dispatch_offsets[1] == 0
)
@constraint(
    model,
    unoptimised_last_trip_constraint,
    dispatch_offsets[num_trips] == 0
)
# holding duration to be at least as long as minimum dwell time
@constraint(
    model,
    holding_duration_minimum_constraint,
    holding_durations[j,s] >= minimum_dwell[j,s] for j in 1:num_trips for s in 1:num_stops
)

# should not exceed bus capacity
@constraint(
    model,
    bus_capacity_constraint,
    num_boarding[j,s] + busload[j,s] - num_alighting[j,s] <= bus_capacity for j in 1:num_trips for s in 1:num_stops
)
# should have sufficient buses to dispatch out
@constraint(
    model,
    sufficient_bus_constraint,
    (dispatch_times[j] + dispatch_offsets[j]) >= arrival_time[j-num_buses, num_stops] for j in num_buses+1:num_trips
)

# OBJECTIVE FUNCTION
@objective(
    model,
    Min,
    sum(stop_weights[s] * (sum((headway[j,s] - target_headway[j,s])^2) for j in 2:num_trips)) for s in 1:num_stops
)
optimize!(model)

println(value.(arrival_time))

println(objective_value(model))

for offsets in dispatch_offsets
    println(value(offsets))
end

for holding_duration in holding_durations
    println(value(holding_duration))
end

LoadError: UndefVarError: `num_want_board` not defined

In [73]:
# trying non-linear formulation

using JuMP
using Ipopt

model = Model(Ipopt.Optimizer)

# PARAMETERS
num_trips = 5 # rows
num_stops = 5 # columns
dispatch_times = [4, 6, 8, 10, 12]

target_headway = [
    1 2 2 1 2
    1 2 2 1 2
    1 2 2 1 2
    1 2 2 1 2
]

interstation_travel_time = [
    1 2 3 4
    1 2 3 4
    1 2 3 4
    1 2 3 4
    1 2 3 4
]

arrival_rate = 0.5
alighting_percentage = [0.1, 0.2, 0.3, 0.4, 0.5]
boarding_time = 1
alighting_time = 2
stop_weights = [0.1, 0.2, 0.3, 0.4, 0.5]
num_buses = 5
bus_capacity = 100
initial_passengers = [0, 0, 0, 0, 0]

# DECISION VARIABLES
@variable(model, dispatch_offsets[1:num_trips] >= 0)
@variable(model, holding_durations[1:num_trips, 1:num_stops] >= 0)

# EXPRESSIONS - Define nonlinear expressions

# arrival time js
@NLexpression(model, arrival_time[j = 1:num_trips, s = 1:num_stops],
    dispatch_times[j] +
    dispatch_offsets[j] +
    sum(interstation_travel_time[j, φ] for φ in 1:s-1) +
    sum(holding_durations[j, φ] for φ in 2:s-1)
)

# headway js
@NLexpression(model, headway[j = 2:num_trips, s = 1:num_stops],
    arrival_time[j, s] - arrival_time[j-1, s]
)

# minimum dwell time js
@NLexpression(model, minimum_dwell[j = 1:num_trips, s = 1:num_stops],
    max((@NLexpression(model, num_boarding[j, s] * boarding_time))^2, (@NLexpression(model, num_alighting[j, s] * alighting_time))^2)
)

# busload js
@NLexpression(model, busload[j = 1:num_trips, s = 1:num_stops],
    busload[j, s-1] - num_alighting[j, s-1] + num_want_board[j, s-1]
)

# number of passengers wanting to board js
@NLexpression(model, num_want_board[j = 1:num_trips, s = 1:num_stops],
    (1 + arrival_rate * boarding_time) * (arrival_rate * (headway[j, s] - minimum_dwell[j-1, s]) + stranded[j-1, s])
)

# number of passengers alighting js
@NLexpression(model, num_alighting[j = 1:num_trips, s = 1:num_stops],
    (round(busload[j, s] * alighting_percentage[s]))^2
)

# stranded passengers js
@NLexpression(model, stranded[j = 1:num_trips, s = 1:num_stops],
    (num_want_board[j, s] - num_boarding[j, s])^2
)

# CONSTRAINTS - Define nonlinear constraints

# no overtaking
@NLconstraint(model, no_overtaking_constraint[j in 1:num_trips, s in 1:num_stops],
    headway[j, s] >= 0
)

# first and last trip cannot be optimized
@constraint(model, unoptimized_first_trip_constraint,
    dispatch_offsets[1] == 0
)

@constraint(model, unoptimized_last_trip_constraint,
    dispatch_offsets[num_trips] == 0
)

# holding duration to be at least as long as minimum dwell time
@NLconstraint(model, holding_duration_minimum_constraint[j in 1:num_trips, s in 1:num_stops],
    holding_durations[j, s] >= minimum_dwell[j, s]
)

# should not exceed bus capacity
@NLconstraint(model, bus_capacity_constraint[j in 1:num_trips, s in 1:num_stops],
    num_boarding[j, s] + busload[j, s] - num_alighting[j, s] <= bus_capacity
)

# should have sufficient buses to dispatch out
@constraint(model, sufficient_bus_constraint[j in num_buses+1:num_trips],
    dispatch_times[j] + dispatch_offsets[j] >= arrival_time[j - num_buses, num_stops]
)

# OBJECTIVE FUNCTION - Define a nonlinear objective function
@NLobjective(model, Min,
    sum(stop_weights[s] * sum(@NLexpression(model, (headway[j, s] - target_headway[j, s])^2) for j in 2:num_trips) for s in 1:num_stops)
)

# Solve the NLP
optimize!(model)


LoadError: Unsupported expression: JuMP.var"@NLexpression" A JuMP Model
Feasibility problem with:
Variables: 30
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 30 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt
Names registered in the model: arrival_time, dispatch_offsets, headway, holding_durations num_boarding[1,1] * 1